In [1]:
import numpy as np
import sys
import geopandas as gpd

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

sys.path.append("/home/nilscp/GIT/")
from rastertools import utils

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Clip DEM and visible image for rayed craters (SLDEM2015)

In [4]:
location_of_craters = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_LUNA24_EQC_filtered.shp"
dem = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_LUNA24.TIF"
orthoimage = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_LUNA24_M144212439_50CM.tiff"
clip_distance = 4.0
output_dir = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped/"
output_dir_ortho = "/home/nilscp/tmp/NAC_DTM_LUNA24/Ortho_clipped/"
identifier_dem = "NACDTM"
identifier_orthoimage = "NAC"
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/"

In [5]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 42/42 [00:50<00:00,  1.19s/it]


In [6]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 42/42 [00:58<00:00,  1.40s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2015, Rayed craters)

In [9]:
location_of_craters = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_LUNA24_EQC_filtered.shp"
scaling_factor = 1.0
dem_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped/"
dem_detrended_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped_detrended/"
craterID = None
overwrite = True
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/"

In [10]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 42/42 [00:17<00:00,  2.43it/s]


## The detection of the rim composite, and the following update of the crater centre
----
#### Note that the algorithm will not work if:
- the centre of the crater is outside of the crater of interest
- the resolution of the dem is too low (at least 20 cells is required along the crater radius to provide reasonable results <br>
----
#### It can be that the algorithm work but gives wrong results if:
- There are large change in slopes that does not correspond to the actual change in slope associated with the rim (either very close or very far)

In [11]:
dem_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped_detrended/"
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/"
suffix = "_NACDTM_detrended.tif"

In [12]:
rim_detection.main(location_of_craters, dem_folder, shp_folder, suffix, threshold_min=None, threshold_max=None , craterID = None)

  0%|          | 0/42 [00:00<?, ?it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
100%|██████████| 42/42 [03:14<00:00,  4.63s/it]


## Manual check of results
It is important to manually check if the detected rims give meaningful results. If not, the polygon can be deleted, and replace by a manually generated ellipse. This should (hopefully) not happen in many cases for fresh impact structures (<5%). However, this is likely to happen more often for degraded craters (such as the rayed crater dataset, that contains both young and old craters). For completion, we are writing down the id of the crater where a problem is detected, to understand if we can improve the rule-based detection of rim in the future. 

## Update centre of craters
- Merge the centres of craters together



In [13]:
out_shapefile = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/final_crater_center/global/fresh_impact_craters_LUNA24_validation_lonlat_manually_checked.shp"
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/polygons/"
old_crater_centres = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_LUNA24_EQC_filtered.shp"

In [17]:
rim_detection.update_crater_centres(shp_folder, out_shapefile, old_crater_centres, replace_crs=False)

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
- decrease clip distance to 4R to divide the size of DEMs by 2 (for SLDEM2015 and SLDEM2013)
- could be that the size of the SLDEM2013 could be decreased by moving away from float.

In [17]:
location_of_craters = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/final_crater_center/global/fresh_impact_craters_LUNA24_validation_lonlat_manually_checked.shp"
clip_distance = 4.0
output_dir = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped/"
output_dir_ortho = "/home/nilscp/tmp/NAC_DTM_LUNA24/Ortho_clipped/"
identifier_dem = "NACDTM"
identifier_orthoimage = "NAC"
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/final_crater_center/"

In [18]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 42/42 [00:50<00:00,  1.21s/it]


In [19]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 42/42 [01:03<00:00,  1.51s/it]


## Update the 2.0R-3.0R and 0.9R-1.1R detrendings

In [20]:
scaling_factor = 1.0
dem_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped/"
dem_detrended_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped_detrended/"
craterID = None
overwrite = True
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/final_crater_center/"

In [21]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 42/42 [00:15<00:00,  2.71it/s]


## Calculate morphometrical parameters from the second ellipse, final crater centre and new detrended products
- load ellipse in world coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r (maybe need to modify with the -xc, yc

There are a few problems by using ellipse... approximating the rim of a crater with an ellipse is totally ok, but it will always result in "shooting" over or under the actual position of the rim of crater. The majority of morphometric parameters (e.g., middle, upper, rim flank slope) are a bit sensitive to the exact location of the rim. I think I will try to create a function that retrieve the breaks in slope within 10% of the ellipse, and then use those few locations to extract cross sections and calculate parameters. This means that not all of the craters will have a constant number of cross-sections (512) as for ellipses. I will compare the results of those different values.

In [ ]:
location_of_craters = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/final_crater_center/global/fresh_impact_craters_LUNA24_validation_lonlat_manually_checked.shp"
shp_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/shapefiles/polygons/"
dem_folder = "/home/nilscp/tmp/NAC_DTM_LUNA24/DTM_clipped_detrended/"
suffix = "_NACDTM_detrended.tif"
prefix = 'crater'
craterID=None